In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import seaborn as sns

%matplotlib inline  

https://interfaithdental.com/smile-on-60/advocate/
Tennessee ranks 49th in senior dental care. 22% of older Tennesseans will lose all their natural teeth. 

SMILE ON 60+ is a statewide initiative with the goal of improving the overall health of low-income, mobile seniors age 60+ through access to oral healthcare services and community education.  SMILE ON 60+ will evaluate, educate, and navigate seniors into dental homes and then transport, treat, and repeat.

The funding for this project was provided through a grant awarded by the Davidson County Chancery Court, Part III from the SeniorTrust/ElderTrust settlement (Case No. 11-1548-III) and through a contract administered by the Tennessee Commission on Aging and Disability.

State Stats: https://nccd.cdc.gov/oralhealthdata/rdPage.aspx?rdReport=DOH_DATA.ExploreByLocation&rdProcessAction=&SaveFileGenerated=1&islLocation=47&rdICL-iclTopic=ADT&iclTopic_rdExpandedCollapsedHistory=&iclTopic=ADT&islYear=2018&hidLocation=47&hidTopic=ADT&hidYear=2018&irbShowFootnotes=Show&rdICL-iclIndicators=ADT1_1%2cADT1_3%2cADT1_4&iclIndicators_rdExpandedCollapsedHistory=&iclIndicators=ADT1_1%2cADT1_3%2cADT1_4&hidPreviouslySelectedIndicators=&DashboardColumnCount=2&rdShowElementHistory=&rdScrollX=0&rdScrollY=0&rdRnd=69609

In [ ]:
#read in county geo data for map later:

counties = gpd.read_file('../data/county/tncounty.shp')

In [ ]:
#counties.to_csv(r'C:\Users\kkosf\Documents\nss\projects\smile-on-spiderman\hardin_county.csv', index=False)
counties.head()

In [ ]:
#read in poverty data for map later and split "County" from county name to merge with other data later:

TN_poverty = pd.read_excel('../data/PovertyEstimates.xls')

MapLocSplit = TN_poverty["Area_name"].str.rsplit(" ", n = 1, expand = True) 
TN_poverty["name"]= MapLocSplit[0] 
TN_poverty["county"]= MapLocSplit[1] 

TN_poverty.tail(10)

In [ ]:
#read in smile on data:

smileon_full = pd.read_csv('../data2/SmileOn11.28.2020.csv', skiprows=2, encoding='ISO-8859-1')

smileon_full.head()

In [ ]:
#beginning EDA for smile on data:

smileon_full.isnull().sum()

In [ ]:
smileon_full.StateProvince.value_counts()

In [ ]:
smileon_full['SMILE ON 60+ Site Information - Enroller Location'].value_counts()

In [ ]:
smileon_full['SMILE ON 60+ Case Management Information - Dental Clinic'].value_counts()

In [ ]:
# create a new dataframe with fewer columns:

smileon = smileon_full[['CallReportNum', 'ReportVersion', 'CallDateAndTimeStart', 'CallerNum', 'CountyName', 'PostalCode', 
                        'SMILE ON 60+ Screening - Last Screening Date', 
                        'SMILE ON 60+ Clinic Information - Is patient of record date if yes last seen?', 
                        'SMILE ON 60+ Clinic Information - Was initial appointment made?',
                        'SMILE ON 60+ Demographic Information - When is the last time you visited your dentist?',
                        'SMILE ON 60+ Case Management Information - What barriers have kept you from finding a consistent dental home? (Select all that apply)',
                        'SMILE ON 60+ Oral Care Encounter - What care was provided to the enrollee? (Select all that apply)',
                        'SMILE ON 60+ Screening - How often do you brush your teeth?', 
                        'SMILE ON 60+ Screening - Need for Periodontal Care',
                        'SMILE ON 60+ Screening - Root Fragments', 
                        'SMILE ON 60+ Screening - Suspicious Soft Tissue Lesions',
                        'SMILE ON 60+ Screening - Treatment Urgency', 
                        'SMILE ON 60+ Screening - Untreated Decay', 
                        'SMILE ON 60+ Site Information - Enroller ID',
                        'SMILE ON 60+ Treatment Plan - Is the treatment plan completed?', 
                        'SMILE ON 60+ Treatment Plan - Was a treatment plan developed?',
                        'SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)'
                       ]]

#and rename the columns to make things easier:

smileon = smileon.rename(columns = {'SMILE ON 60+ Screening - Last Screening Date': 'Last_Screen_Date', 
                                    'SMILE ON 60+ Clinic Information - Is patient of record date if yes last seen?' :'Date_Last_Seen', 
                                    'SMILE ON 60+ Clinic Information - Was initial appointment made?' :'Initial_Appt_Made',
                                    'SMILE ON 60+ Demographic Information - When is the last time you visited your dentist?': 'Last_Dentist_Visit',
                                    'SMILE ON 60+ Case Management Information - What barriers have kept you from finding a consistent dental home? (Select all that apply)' : 'Barriers',
                                    'SMILE ON 60+ Oral Care Encounter - What care was provided to the enrollee? (Select all that apply)' : 'Care_Provided',
                                    'SMILE ON 60+ Screening - How often do you brush your teeth?' : 'Brush_Frequency', 
                                    'SMILE ON 60+ Screening - Need for Periodontal Care' : 'Need_For_Care',
                                    'SMILE ON 60+ Screening - Root Fragments' : 'Root_Fragments', 
                                    'SMILE ON 60+ Screening - Suspicious Soft Tissue Lesions' : 'Lesions',
                                    'SMILE ON 60+ Screening - Treatment Urgency' : 'Urgency', 
                                    'SMILE ON 60+ Screening - Untreated Decay' : 'Decay', 
                                    'SMILE ON 60+ Site Information - Enroller ID' : 'Clinic_Attended',
                                    'SMILE ON 60+ Treatment Plan - Is the treatment plan completed?' : 'Tx_Plan_Complete', 
                                    'SMILE ON 60+ Treatment Plan - Was a treatment plan developed?' : 'Tx_Plan_Developed',
                                    'SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)' :
                                    'Function_Restored'
                                                
                                   })

smileon.head()

In [ ]:
smileon.isnull().sum()

In [ ]:
smileon.CallerNum.nunique()

In [ ]:
smileon.Last_Dentist_Visit.value_counts()

In [ ]:
smileon.CallerNum.value_counts()

In [ ]:
#remove -1 & -2 from CallerNum as suggested:

smileon = smileon.loc[~smileon['CallerNum'].isin(['-1', '-2'])]
smileon.CallerNum.value_counts()

In [ ]:
# change CallDateAndTimeStart to datetime and separate date and time:

smileon['CallDateAndTimeStart'] = pd.to_datetime(smileon['CallDateAndTimeStart'])
smileon['date'] = smileon['CallDateAndTimeStart'].dt.date
smileon['time'] = smileon['CallDateAndTimeStart'].dt.time

smileon.head(3)

In [ ]:
#Further EDA:

smileon.ReportVersion.value_counts()

#over twice as many clinical than registration (difference of 14,832)

In [ ]:
smileon.Brush_Frequency.value_counts()

In [ ]:
smileon.CountyName.value_counts()

In [ ]:
smileon.PostalCode.value_counts()

In [ ]:
# Checking the number of clinics and how many times they were used:

print(smileon['Clinic_Attended'].nunique())

smileon['Clinic_Attended'].value_counts()

In [ ]:
smileon.Decay.value_counts()

In [ ]:
smileon.Root_Fragments.value_counts()

In [ ]:
smileon.Lesions.value_counts()

In [ ]:
smileon.Tx_Plan_Complete.value_counts()

In [ ]:
smileon.Tx_Plan_Developed.value_counts()

In [ ]:
smileon.Function_Restored.value_counts()

In [ ]:
smileon.date.value_counts()

In [ ]:
smileon.CallerNum.value_counts().head(20)

In [ ]:
#Group CallerNums to look at those with many entries:

CallerNums = smileon.groupby('CallerNum')

In [ ]:
CallerNums.get_group(2441776)

In [ ]:
CallerNums.get_group(2416076)

In [ ]:
# remove duplicates from data to look at individuals:

no_dups = smileon.drop_duplicates(subset='CallerNum', keep="first")
no_dups.info()

In [ ]:
# Create a dataset with just county name and total number of users. Also had to make lowercase to merge with other data later:

no_dups['CountyName_lower'] = no_dups['CountyName'].str.lower()
county_counts = no_dups.CountyName_lower.value_counts()

county_counts = county_counts.to_frame().reset_index()

county_counts = county_counts.rename(columns = {'index': 'CountyName', 'CountyName_lower': 'total'})
county_counts.sort_values(by='CountyName',inplace=True)
#county_counts.to_csv(r'C:\Users\kkosf\Documents\nss\projects\smile-on-spiderman\county_counts.csv', index=False)
print(county_counts.shape)

In [ ]:
# Merge county geo data with poverty data in order to create poverty map by county:

pov_map = pd.merge(left = counties, right = TN_poverty, 
                    left_on = 'NAME', right_on = 'name')
print(pov_map.shape)
pov_map.head(3)

In [ ]:
#Create map of poverty percent by county:

fig, ax = plt.subplots(figsize=(20,6))
pov_map.plot(column = 'PCTPOVALL_2018', 
              cmap = 'GnBu', 
              edgecolor = 'black', 
              legend = True,
              ax = ax)

for index, row in pov_map.iterrows():
    plt.annotate(text=row['PCTPOVALL_2018'], 
                 xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                 horizontalalignment='center', fontweight = 'bold', color = 'tomato')

plt.title('Percent of Poverty by County, 2018', fontsize = 14)
ax.axis('off');

In [ ]:
#Merge smile on county totals with county geo data to create users by county:

county_counts = pd.read_csv('../data/county_counts.csv')

counties['NAME_lower'] = counties['NAME'].str.lower()

smileon_map = pd.merge(left = counties, right = county_counts, 
                    left_on = 'NAME_lower', right_on = 'CountyName')
print(smileon_map.shape)
smileon_map.head(3)

In [ ]:
pts_by_county = smileon_map[['NAME', 'geometry', 'total']]
pts_by_county = pts_by_county.rename(columns ={'NAME' : 'County', 'total' : 'patients'})
pts_by_county.sort_values('County', ignore_index=True, inplace=True)
print(pts_by_county.head())
#pts_by_county.to_csv(r'C:\Users\kkosf\Documents\nss\projects\smile-on-spiderman\pts_by_county.csv', index=False)

In [ ]:
#Create map of smile on users by county:

fig, ax = plt.subplots(figsize=(20,6))
smileon_map.plot(column = 'total', 
              cmap = 'GnBu', 
              edgecolor = 'black', 
              legend = True,
              ax = ax)

for index, row in smileon_map.iterrows():
    plt.annotate(text=row['total'], 
                 xy=(row['geometry'].centroid.x, row['geometry'].centroid.y),
                 horizontalalignment='center', fontweight = 'bold', color = 'tomato')
    
plt.title('Number of Smile On 60+ Users per County', fontsize = 14)
ax.axis('off');

# NOTE: no users from Hardin county

In [ ]:
#number of users by county table with no geography:

pts_by_county_simple = pts_by_county[['County', 'patients']]
#pts_by_county_simple.to_csv(r'C:\Users\kkosf\Documents\nss\projects\smile-on-spiderman\pts_by_county_simple.csv', index=False)
pts_by_county_simple

In [ ]:
# Create a dataframe with less info and remove day from date to get year-month only to make some plots over time:

use_over_time = no_dups[['ReportVersion', 'CallReportNum', 'date']].copy() 
use_over_time.date = use_over_time.date.apply(pd.to_datetime)
use_over_time.date = use_over_time.date.dt.strftime('%Y-%m')
print(use_over_time.shape)
use_over_time.sort_values(by='date',inplace=True)
use_over_time.head()

In [ ]:
#plot of users over time, still needs work...

plt.plot('date', 'CallReportNum', data = use_over_time)
plt.xticks(rotation = 70)
plt.title('Use Over Time');

In [ ]:
#separate out registration vs clinical:

reg_only = smileon[smileon['ReportVersion'] == 'SMILE ON 60+ Registration'] 
print(reg_only.shape)

clinic_only =  smileon[smileon['ReportVersion'] == 'SMILE ON 60+ Clinical'] 
print(clinic_only.shape)
clinic_only.head()

In [ ]:
clinic_only.info() 

In [ ]:
#make smaller dataframe to use for a plot:

clinic_over_time = clinic_only[['ReportVersion', 'CallReportNum', 'date']].copy() 
clinic_over_time.date = clinic_over_time.date.apply(pd.to_datetime)
clinic_over_time.date = clinic_over_time.date.dt.strftime('%Y-%m')
print(clinic_over_time.shape)
clinic_over_time.sort_values(by='date',inplace=True)
clinic_over_time.head()

In [ ]:
#create plot with only clinic data

plt.plot('date', 'CallReportNum', data = clinic_over_time)
plt.xticks(rotation = 70)
plt.title('Use Over Time - Clinic');

In [ ]:
no_dups.Last_Dentist_Visit.value_counts(normalize=True)

#clinic_only
#Less than 12 months    0.481960
#More than 5 years      0.201395
#1 to 2 years           0.188389
#3 to 5 years           0.128256

In [ ]:
Last_visit = no_dups.Last_Dentist_Visit.value_counts().to_frame('counts').reset_index()
Last_visit.rename(columns = {'index' : 'Last_Visit'})
#Last_visit.to_csv(r'C:\Users\kkosf\Documents\nss\projects\smile-on-spiderman\Last_visit.csv', index=False)
Last_visit

In [ ]:
sns.set_palette("ch:s=.25,rot=-.25")
plt.figure(figsize=(12,6))
sns.countplot(x="Last_Dentist_Visit", data = no_dups, 
              order=['Less than 12 months', '1 to 2 years', '3 to 5 years', 'More than 5 years'])
plt.xlabel("")
plt.ylabel("")
plt.title("Last Dentist Visit");

In [ ]:
#create a dataframe for people who haven't been to the dentist in more than 5 years:

five_plus = smileon[smileon['Last_Dentist_Visit'] == 'More than 5 years'] 
print(five_plus.shape)
five_plus.ReportVersion.value_counts()

In [ ]:
five_plus.info()

In [ ]:
five_plus.Brush_Frequency.value_counts()

In [ ]:
five_plus_clinic = clinic_only[clinic_only['Last_Dentist_Visit'] == 'More than 5 years'] 
five_plus_clinic.shape

In [ ]:
five_plus_clinic.CallerNum.value_counts().head(20)

In [ ]:
five_plus_clinic.CallerNum.nunique()

In [ ]:
five_plus_clinic.info()

In [ ]:
five_plus_grouped_fx = five_plus_clinic.groupby(['CallerNum']).agg({'Function_Restored': "count"}).reset_index()
five_plus_grouped_fx

In [ ]:
five_plus_grouped_fx.Function_Restored.value_counts()

#219 of 1102 of those that haven't been to the dentist in 5 years do not have function restored - 20%


In [ ]:
219/1102

In [ ]:
five_plus_grouped_txcomp = five_plus_clinic.groupby(['CallerNum']).agg({'Tx_Plan_Complete': "count"}).reset_index()
five_plus_grouped_txcomp

In [ ]:
five_plus_grouped_txcomp.Tx_Plan_Complete.value_counts()

#144 of 1102 of those that haven't been to the dentist in 5 years do not have their treatment plan complete - 13%


In [ ]:
144/1102

In [ ]:
five_plus_clinic.Barriers.value_counts().head(10)

In [ ]:
#look at the top barriers to dental visits for all users:

top_barriers = no_dups.Barriers.value_counts().head(10).to_frame().reset_index()
top_barriers = top_barriers.rename(columns = {'index': 'Barriers', 'Barriers': 'count'})
top_barriers

In [ ]:
ax=top_barriers.sort_values(by='count').plot.barh(x='Barriers', figsize=(10, 5))
ax.get_legend().remove()
plt.ylabel("")
plt.title('Top 10 Barriers');

In [ ]:
care_provided = clinic_only[['CallReportNum', 'CallerNum', 'CountyName', 'Care_Provided',
                        'Clinic_Attended', 'Tx_Plan_Complete', 'Tx_Plan_Developed', 'Function_Restored', 'date'
                       ]]

care_provided.head()

In [ ]:
care_provided.Function_Restored.value_counts(normalize=True)

In [ ]:
care_provided.Tx_Plan_Complete.value_counts(normalize=True)

In [ ]:
care_provided.Tx_Plan_Developed.value_counts(normalize=True)

In [ ]:
care_provided.CallerNum.value_counts().tail(10)

In [ ]:
CareInfo = care_provided.groupby('CallerNum')
CareInfo.get_group(3558797)

In [ ]:
# Working on subsetting people that only had one visit - look for those with tx plan not completed plus barriers?

one_visit = smileon.CallerNum.value_counts().loc[lambda x : x==1]
one_visit